# Using ampute from MICE

### To generate the missing values, the method data_ampute bellow needs the following arguments

X: the original complete dataset

p_miss: proportion of missing values to generate for variables which will have missing values

mechanism: "MCAR", "MAR", or "MNAR" 

In [ ]:
!pip install rpy2
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import pandas2ri
import rpy2.robjects as ro

import warnings 
warnings.filterwarnings("ignore")

def data_ampute(X, prop, mech):
    #r_ampute = importr('mice')
    with localconverter(ro.default_converter + pandas2ri.converter):
      r_from_pd_df = ro.conversion.py2rpy(X)
    
    result = ro.globalenv['ampute']
    data = result(r_from_pd_df, prop = prop, mech = mech) #Run ampute
    pandas2ri.activate() #pandas conversion is activated
    data = ro.conversion.rpy2py(data) #convert r dataframe to python dataframe

    return data


In [ ]:
#Check https://rdrr.io/cran/mice/src/R/ampute.R
import rpy2.robjects as ro
ro.r('''

ampute <- function(data, prop = 0.5, patterns = NULL, freq = NULL,
                   mech = "MAR", weights = NULL, std = TRUE, cont = TRUE,
                   type = NULL, odds = NULL,
                   bycases = TRUE, run = TRUE) {
 
  #Remove the first column
  #data_first_column <- select(data,1)
  #data$select(data,1) <- NULL
  ##############################
  
  if (is.null(data)) {
    stop("Argument data is missing, with no default", call. = FALSE)
  }
  data <- check.dataform(data)
  if (anyNA(data)) {
    stop("Data cannot contain NAs", call. = FALSE)
  }
  if (ncol(data) < 2) {
    stop("Data should contain at least two columns", call. = FALSE)
  }
  data <- data.frame(data)
  if (any(vapply(data, Negate(is.numeric), logical(1))) && mech != "MCAR") {
    data <- as.data.frame(sapply(data, as.numeric))
    warning("Data is made numeric because the calculation of weights requires numeric data",
      call. = FALSE
    )
  }
  if (prop < 0 || prop > 100) {
    stop("Proportion of missingness should be a value between 0 and 1 (for a proportion) or between 1 and 100 (for a percentage)",
      call. = FALSE
    )
  } else if (prop > 1) {
    prop <- prop / 100
  }
  if (is.null(patterns)) {
    patterns <- ampute.default.patterns(n = ncol(data))
  } else if (is.vector(patterns) && (length(patterns) / ncol(data)) %% 1 == 0) {
    patterns <- matrix(patterns, length(patterns) / ncol(data), byrow = TRUE)
    if (nrow(patterns) == 1 && all(patterns[1, ] %in% 1)) {
      stop("One pattern with merely ones results to no amputation at all, the procedure is therefore stopped", call. = FALSE)
    }
  } else if (is.vector(patterns)) {
    stop("Length of pattern vector does not match #variables", call. = FALSE)
  }
  patterns <- data.frame(patterns)
  
  if (is.null(freq)) {
    freq <- ampute.default.freq(patterns = patterns)
  }
  if (!is.vector(freq)) {
    freq <- as.vector(freq)
    warning("Frequency should be a vector", call. = FALSE)
  }
  if (length(freq) != nrow(patterns)) {
    if (length(freq) > nrow(patterns)) {
      freq <- freq[seq_along(nrow(patterns))]
    } else {
      freq <- c(freq, rep.int(0.2, nrow(patterns) - length(freq)))
    }
    warning(paste("Length of vector with relative frequencies does not match #patterns and is therefore changed to", freq), call. = FALSE)
  }
  if (sum(freq) != 1) {
    freq <- recalculate.freq(freq = freq)
  }
  if (!bycases) {
    prop <- recalculate.prop(
      prop = prop,
      freq = freq,
      patterns = patterns,
      n = ncol(data)
    )
  }
  check.pat <- check.patterns(
    patterns = patterns,
    freq = freq,
    prop = prop
  )
  patterns.new <- check.pat[["patterns"]]
  freq <- check.pat[["freq"]]
  prop <- check.pat[["prop"]]
  if (any(!mech %in% c("MCAR", "MAR", "MNAR"))) {
    stop("Mechanism should be either MCAR, MAR or MNAR", call. = FALSE)
  }
  if (!is.vector(mech)) {
    mech <- as.vector(mech)
    warning("Mechanism should contain merely MCAR, MAR or MNAR", call. = FALSE)
  } else if (length(mech) > 1) {
    mech <- mech[1]
    warning("Mechanism should contain merely MCAR, MAR or MNAR. First element is used",
      call. = FALSE
    )
  }
  # Check if there is a pattern with merely zeroos
  if (!is.null(check.pat[["row.zero"]]) && mech == "MAR") {
    stop(paste("Patterns object contains merely zeros and this kind of pattern is not possible when mechanism is MAR"),
      call. = FALSE
    )
  }
  if (mech == "MCAR" && !is.null(weights)) {
    weights <- NULL
    warning("Weights matrix is not used when mechanism is MCAR", call. = FALSE)
  }
  if (mech == "MCAR" && !is.null(odds)) {
    odds <- NULL
    warning("Odds matrix is not used when mechanism is MCAR", call. = FALSE)
  }
  if (mech != "MCAR" && !is.null(weights)) {
    if (is.vector(weights) && (length(weights) / ncol(data)) %% 1 == 0) {
      weights <- matrix(weights, length(weights) / ncol(data), byrow = TRUE)
    } else if (is.vector(weights)) {
      stop("Length of weight vector does not match #variables", call. = FALSE)
    } else if (!is.matrix(weights) && !is.data.frame(weights)) {
      stop("Weights matrix should be a matrix", call. = FALSE)
    }
  }
  if (is.null(weights)) {
    weights <- ampute.default.weights(
      patterns = patterns.new,
      mech = mech
    )
  }
  weights <- as.data.frame(weights)
  if (!nrow(weights) %in% c(nrow(patterns), nrow(patterns.new))) {
    stop("The objects patterns and weights are not matching", call. = FALSE)
  }
  if (!is.vector(cont)) {
    cont <- as.vector(cont)
    warning("Continuous should contain merely TRUE or FALSE", call. = FALSE)
  } else if (length(cont) > 1) {
    cont <- cont[1]
    warning("Continuous should contain merely TRUE or FALSE. First element is used",
      call. = FALSE
    )
  }
  if (!is.logical(cont)) {
    stop("Continuous should contain TRUE or FALSE", call. = FALSE)
  }
  if (cont && !is.null(odds)) {
    odds <- NULL
    warning("Odds matrix is not used when continuous probabilities (cont == TRUE) are specified",
      call. = FALSE
    )
  }
  if (!cont && !is.null(type)) {
    type <- NULL
    warning("Type is not used when discrete probabilities (cont == FALSE) are specified",
      call. = FALSE
    )
  }
  if (is.null(type)) {
    type <- ampute.default.type(patterns = patterns.new)
  }
  if (any(!type %in% c("LEFT", "MID", "TAIL", "RIGHT"))) {
    stop("Type should contain LEFT, MID, TAIL or RIGHT",
      call. = FALSE
    )
  }
  if (!is.vector(type)) {
    type <- as.vector(type)
    warning("Type should be a vector of strings", call. = FALSE)
  } else if (!length(type) %in% c(1, nrow(patterns), nrow(patterns.new))) {
    type <- type[1]
    warning("Type should either have length 1 or length equal to #patterns, first element is used for all patterns", call. = FALSE)
  }
  if (mech != "MCAR" && !is.null(odds) && !is.matrix(odds)) {
    if (nrow(patterns.new) == 1 && is.vector(odds)) {
      odds <- matrix(odds, nrow = 1)
    } else {
      stop("Odds matrix should be a matrix", call. = FALSE)
    }
  }
  if (is.null(odds)) {
    odds <- ampute.default.odds(patterns = patterns.new)
  }
  if (!cont) {
    for (h in seq_len(nrow(odds))) {
      if (any(!is.na(odds[h, ]) & odds[h, ] < 0)) {
        stop("Odds matrix can only have positive values", call. = FALSE)
      }
    }
  }
  if (!nrow(odds) %in% c(nrow(patterns), nrow(patterns.new))) {
    stop("The objects patterns and odds are not matching", call. = FALSE)
  }
  #
  # Start using arguments
  # Create empty objects
  P <- NULL
  scores <- NULL
  missing.data <- NULL
  # Apply function (run = TRUE) or merely return objects (run = FALSE)
  if (run) {
    # Assign cases to the patterns according probs
    # Because 0 and 1 will be used for missingness,
    # the numbering of the patterns will start from 2
    P <- sample.int(
      n = nrow(patterns.new), size = nrow(data),
      replace = TRUE, prob = freq
    ) + 1
    # Calculate missingness according MCAR or calculate weighted sum scores
    # Standardized data is used to calculate weighted sum scores
    if (mech == "MCAR") {
      R <- ampute.mcar(
        P = P,
        patterns = patterns.new,
        prop = prop
      )
    } else {
      scores <- sum.scores(
        P = P,
        data = data,
        std = std,
        weights = weights,
        patterns = patterns
      )
      if (!cont) {
        R <- ampute.discrete(
          P = P,
          scores = scores,
          odds = odds,
          prop = prop
        )
      } else if (cont) {
        R <- ampute.continuous(
          P = P,
          scores = scores,
          prop = round(prop, 3),
          type = type
        )
      }
    }
    missing.data <- data
    for (i in seq_len(nrow(patterns.new))) {
      if (any(P == (i + 1))) {
        missing.data[R[[i]] == 0, patterns.new[i, ] == 0] <- NA
      }
    }
  }
  #
  # Create return object
  names(patterns.new) <- names(data)
  names(weights) <- names(data)
  call <- match.call()
  missing.data <- data.frame(missing.data)
  result <- list(
    call = call,
    prop = prop,
    patterns = patterns.new,
    freq = freq,
    mech = mech,
    weights = weights,
    cont = cont,
    std = std,
    type = type,
    odds = odds,
    amp = missing.data,
    cand = P - 1,
    scores = scores,
    data = as.data.frame(data)
  )
  #
  # Return result
  oldClass(result) <- "mads"
  #result
  missing.data #This return was modified to get just the dataframe
}

# This is an underlying function of multivariate amputation function ampute().
# This function is used to calculate the weighted sum scores of the candidates.
# Based on the data, the weights matrix and the kind of mechanism, each case
# will obtain a certain score that will define his probability to be made missing.
# The calculation of the probabilities occur in the function ampute.mcar(),
# ampute.continuous() or ampute.discrete(), based on the kind of missingness.
sum.scores <- function(P, data, std, weights, patterns) {
  weights <- as.matrix(weights)
  f <- function(i) {
    if (length(P[P == (i + 1)]) == 0) {
      return(0) # this will ensure length 1 which is used in ampute.continuous
    } else {
      candidates <- as.matrix(data[P == (i + 1), ])
      # For each candidate in the pattern, a weighted sum score is calculated
      if (std) {
        length_unique <- function(x) {
          return(length(unique(x)) == 1)
        }
        # shangzhi-hong, Feb 2020, #216
        if (nrow(candidates) > 1 && !(any(apply(candidates, 2, length_unique)))) {
          candidates <- scale(candidates)
        }
      }
      scores <- apply(candidates, 1, function(x) weights[i, ] %*% x)
      if (length(scores) > 1 && length(unique(scores)) != 1) {
        scores <- scale(scores)
      }
      return(scores)
    }
  }
  lapply(seq_len(nrow(patterns)), f)
}


#
# ------------------------ recalculate.prop -----------------------------
#
recalculate.prop <- function(prop, n, patterns, freq) {
    # This is an underlying function of multivariate amputation function ampute().
    # The function recalculates the proportion of missing cases for the desired
    # #missing cells.
    miss <- prop * n ^ 2    # Desired #missing cells
    # Calculate #cases according prop and #zeros in patterns
    cases <- vapply(seq_len(nrow(patterns)),
                    function(i)
                        (miss * freq[i]) / length(patterns[i, ][patterns[i, ] == 0]),
                    numeric(1))
    if (sum(cases) > n) {
        #print(sum(cases))
        #print(n)
        stop(
            "Proportion of missing cells is too large in combination with the desired number of missing variables",
            call. = FALSE
        )
    } else {
        prop <- sum(cases) / n
    }
    return(prop)
    
}
#


# This is an underlying function of multivariate amputation function ampute().
# The function recalculates the frequency vector to make the sum equal to 1.
recalculate.freq <- function(freq) {
  freq / sum(freq)
}


# This is an underlying function of multivariate amputation function ampute().
# The function checks whether there are patterns with merely ones or zeroos.
# In case of the first, these patterns will be removed, and argument prop
# and freq will be changed. In case there is a pattern with merely zeroos,
# this is ascertained and saved in the object row.zero.
check.patterns <- function(patterns, freq, prop) {
  prop.one <- 0
  row.one <- c()
  for (h in seq_len(nrow(patterns))) {
    if (any(!patterns[h, ] %in% c(0, 1))) {
      stop(paste("Argument patterns can only contain 0 and 1, pattern", h, "contains another element"), call. = FALSE)
    }
    if (all(patterns[h, ] %in% 1)) {
      prop.one <- prop.one + freq[h]
      row.one <- c(row.one, h)
    }
  }
  if (prop.one != 0) {
    warning(paste("Proportion of missingness has changed from", prop, "to", (1-prop.one)*prop, "because of pattern(s) with merely ones"), call. = FALSE)
    prop <- (1-prop.one)*prop
    freq <- freq[-row.one]
    freq <- recalculate.freq(freq)
    patterns <- patterns[-row.one, ]
    warning("Frequency vector and patterns matrix have changed because of pattern(s) with merely ones", call. = FALSE)
  }
  prop.zero <- 0
  row.zero <- c()
  for (h in seq_len(nrow(patterns))) {
    if (all(patterns[h, ] %in% 0)) {
      prop.zero <- prop.zero + freq[h]
      row.zero <- c(row.zero, h)
    }
  }
  objects <- list(
    patterns = patterns,
    prop = prop,
    freq = freq,
    row.one = row.one,
    row.zero = row.zero
  )
  objects
}

check.data <- function(data, method) {
  check.dataform(data)
}


check.dataform <- function(data) {
  if (!(is.matrix(data) || is.data.frame(data))) {
    stop("Data should be a matrix or data frame", call. = FALSE)
  }
  if (ncol(data) < 2) {
    stop("Data should contain at least two columns", call. = FALSE)
  }
  data <- as.data.frame(data)
  mat <- sapply(data, is.matrix)
  df <- sapply(data, is.data.frame)
  if (any(mat)) {
    stop(
      "Cannot handle columns with class matrix: ",
      colnames(data)[mat]
    )
  }
  if (any(df)) {
    stop(
      "Cannot handle columns with class data.frame: ",
      colnames(data)[df]
    )
  }

  dup <- duplicated(colnames(data))
  if (any(dup)) {
    stop(
      "Duplicate names found: ",
      paste(colnames(data)[dup], collapse = ", ")
    )
  }
  data
}


check.m <- function(m) {
  m <- m[1L]
  if (!is.numeric(m)) {
    stop("Argument m not numeric", call. = FALSE)
  }
  m <- floor(m)
  if (m < 1L) {
    stop("Number of imputations (m) lower than 1.", call. = FALSE)
  }
  m
}


check.cluster <- function(data, predictorMatrix) {
  # stop if the cluster variable is a factor
  isclassvar <- apply(predictorMatrix == -2, 2, any)
  for (j in colnames(predictorMatrix)) {
    if (isclassvar[j] && lapply(data, is.factor)[[j]]) {
      stop("Convert cluster variable ", j, " to integer by as.integer()")
    }
  }
  TRUE
}

check.ignore <- function(ignore, data) {
  if (is.null(ignore)) {
    return(rep(FALSE, nrow(data)))
  }
  if (!is.logical(ignore)) {
    stop("Argument ignore not a logical.")
  }
  if (length(ignore) != nrow(data)) {
    stop(
      "length(ignore) (", length(ignore),
      ") does not match nrow(data) (", nrow(data), ")."
    )
  }
  if (sum(!ignore) < 10L) {
    warning(
      "Fewer than 10 rows for fitting the imputation model. Are you sure?",
      call. = FALSE
    )
  }
  ignore
}

check.newdata <- function(newdata, data) {
  if (is.null(newdata)) {
    stop("No newdata found.")
  }
  if (!is.data.frame(newdata)) {
    stop("newdata not a data.frame.")
  }
  newdata
}

#' Default \code{patterns} in \code{ampute}
#'
#' This function creates a default pattern matrix for the multivariate
#' amputation function \code{ampute()}.
#'
#' @param n A scalar specifying the number of variables in the data.
#' @return A square matrix of size \code{n} where \code{0} indicates a variable
#  should have missing values and \code{1} indicates a variable should remain
#  complete. Each pattern has missingness on one variable only.
#' @seealso \code{\link{ampute}}, \code{\link{md.pattern}}
#' @author Rianne Schouten, 2016
#' @keywords internal
#' @export
ampute.default.patterns <- function(n,p) {
  patterns.list <- lapply(
    seq_len(n),
    function(i) c(rep.int(1, i - 1), 0, rep.int(1, n - i))
  )
  do.call(rbind, patterns.list)
}

#' Default \code{freq} in \code{ampute}
#'
#' Defines the default relative frequency vector for the multivariate
#' amputation function \code{ampute}.
#'
#' @param patterns A matrix of size #patterns by #variables where \code{0} indicates
#' a variable should have missing values and \code{1} indicates a variable should
#' remain complete. Could be the result of \code{\link{ampute.default.patterns}}.
#' @return A vector of length #patterns containing the relative frequencies with
#' which the patterns should occur. An equal probability is given to each pattern.
#' @seealso \code{\link{ampute}}, \code{\link{ampute.default.patterns}}
#' @author Rianne Schouten, 2016
#' @keywords internal
#' @export
ampute.default.freq <- function(patterns) {
  rep.int(1 / nrow(patterns), nrow(patterns))
}

#' Default \code{weights} in \code{ampute}
#'
#' Defines the default weights matrix for the multivariate amputation function
#' \code{ampute}.
#'
#' @param patterns A matrix of size #patterns by #variables where \code{0} indicates
#' a variable should have missing values and \code{1} indicates a variable should
#' remain complete. Could be the result of \code{\link{ampute.default.patterns}}.
#' @param mech A string specifying the missingness mechanism.
#' @return A matrix of size #patterns by #variables containing the weights that
#' will be used to calculate the weighted sum scores. Equal weights are given to
#' all variables. When mechanism is MAR, variables that will be amputed will be
#' weighted with \code{0}. If it is MNAR, variables that will be observed
#' will be weighted with \code{0}. If mechanism is MCAR, the weights matrix will
#' not be used. A default MAR matrix will be returned.
#' @seealso \code{\link{ampute}}, \code{\link{ampute.default.patterns}}
#' @author Rianne Schouten, 2016
#' @keywords internal
#' @export
ampute.default.weights <- function(patterns, mech) {
  weights <- matrix(data = 1, nrow = nrow(patterns), ncol = ncol(patterns))
  if (mech != "MNAR") {
    weights <- matrix(data = 1, nrow = nrow(patterns), ncol = ncol(patterns))
    weights[patterns == 0] <- 0
  } else {
    weights <- matrix(data = 0, nrow = nrow(patterns), ncol = ncol(patterns))
    weights[patterns == 0] <- 1
  }
  weights
}

#' Default \code{type} in \code{ampute()}
#'
#' Defines the default type vector for the multivariate amputation function
#' \code{ampute}.
#'
#' @param patterns A matrix of size #patterns by #variables where 0 indicates a
#' variable should have missing values and 1 indicates a variable should remain
#' complete. Could be the result of \code{\link{ampute.default.patterns}}.
#' @return A string vector of length #patterns containing the missingness types.
#' Each pattern will be amputed with a "RIGHT" missingness.
#' @seealso \code{\link{ampute}}, \code{\link{ampute.default.patterns}}
#' @author Rianne Schouten, 2016
#' @keywords internal
#' @export
ampute.default.type <- function(patterns) {
  rep.int("RIGHT", nrow(patterns))
}

#' Default \code{odds} in \code{ampute()}
#'
#' Defines the default odds matrix for the multivariate amputation function
#' \code{ampute}.
#'
#' @param patterns A matrix of size #patterns by #variables where 0 indicates a
#' variable should have missing values and 1 indicates a variable should remain
#' complete. Could be the result of \code{\link{ampute.default.patterns}}.
#' @return A matrix where #rows equals #patterns. Default is 4 quantiles with odds
#' values 1, 2, 3 and 4, for each pattern, imitating a RIGHT type of missingness.
#' @seealso \code{\link{ampute}}, \code{\link{ampute.default.patterns}}
#' @author Rianne Schouten, 2016
#' @keywords internal
#' @export
ampute.default.odds <- function(patterns) {
  matrix(seq_len(4), nrow = nrow(patterns), ncol = 4, byrow = TRUE)
}



#AMPUTE MCAR

#' Multivariate amputation under a MCAR mechanism
#'
#' This function creates a missing data indicator for each pattern, based on a MCAR
#' missingness mechanism. The function is used in the multivariate amputation function
#' \code{\link{ampute}}.
#'
#' @param P A vector containing the pattern numbers of the cases' candidates.
#' For each case, a value between 1 and #patterns is given. For example, a
#' case with value 2 is candidate for missing data pattern 2.
#' @param patterns A matrix of size #patterns by #variables where \code{0} indicates
#' a variable should have missing values and \code{1} indicates a variable should
#' remain complete. The user may specify as many patterns as desired. One pattern
#' (a vector) is also possible. Could be the result of \code{\link{ampute.default.patterns}},
#' default will be a square matrix of size #variables where each pattern has missingness
#' on one variable only.
#' @param prop A scalar specifying the proportion of missingness. Should be a value
#' between 0 and 1. Default is a missingness proportion of 0.5.
#' @return A list containing vectors with \code{0} if a case should be made missing
#' and \code{1} if a case should remain complete. The first vector refers to the
#' first pattern, the second vector to the second pattern, etcetera.
#' @author Rianne Schouten, 2016
#' @seealso \code{\link{ampute}}
#' @keywords internal
#' @export
ampute.mcar <- function(P, patterns, prop) {
  f <- function(i) {
    # If there are no candidates in a certain pattern, the list will receive a 0
    if (length(P[P == (i + 1)]) == 0) {
      return(0)
    } else {
      # Otherwise, for all candidates in the pattern, the total proportion of
      # missingness is used to define the probabilities to be missing.
      nf <- length(P[P == (i + 1)])
      R.temp <- 1 - rbinom(n = nf, size = 1, prob = prop)
      # Based on the probabilities, each candidate will receive a missing data
      # indicator 0, meaning he will be made missing or missing data indicator 1,
      # meaning the candidate will remain complete.
      R.temp <- replace(P, P == (i + 1), R.temp)
      R.temp <- replace(R.temp, P != (i + 1), 1)
      return(R.temp)
    }
  }

  lapply(seq_len(nrow(patterns)), f)
}

ampute.continuous <- function(P, scores, prop, type) {
  # For a test data set, the shift of the logit function is calculated
  # in order to obtain the right proportion of missingness (area beneath the curve)
  # The set-up for this is created in subsequent lines, it is executed within
  # the for loop over i.
  testset <- scale(rnorm(n = 10000, mean = 0, sd = 1))
  logit <- function(x) exp(x) / (1 + exp(x))
  # An empty list is created, type argument is given the right length
  R <- vector(mode = "list", length = length(scores))
  if (length(type) == 1) {
    type <- rep.int(type, length(scores))
  }
  for (i in seq_along(scores)) {
    # The desired function is chosen
    formula <- switch(type[i],
      LEFT = function(x, b) logit(mean(x) - x + b),
      MID = function(x, b) logit(-abs(x - mean(x)) + 0.75 + b),
      TAIL = function(x, b) logit(abs(x - mean(x)) - 0.75 + b),
      function(x, b) logit(-mean(x) + x + b)
    )
    shift <- bin.search(
      fun = function(shift) {
        sum(formula(x = testset, b = shift)) / length(testset)
      },
      target = prop
    )$where
    if (length(shift) > 1) {
      shift <- shift[1]
    }
    scores.temp <- scores[[i]]
    if (length(scores.temp) == 1 && scores.temp == 0) {
      R[[i]] <- 0
    } else {
      if (length(scores.temp) == 1) {
        probs <- prop
      } else if (length(unique(scores.temp)) == 1) {
        probs <- prop
      } else {
        probs <- formula(x = scores.temp, b = shift)
      }

      # Based on the probabilities, each candidate will receive a missing data
      # indicator 0, meaning he will be made missing or missing data indicator 1,
      # meaning the candidate will remain complete.

      R.temp <- 1 - rbinom(n = length(scores.temp), size = 1, prob = probs)
      R[[i]] <- replace(P, P == (i + 1), R.temp)
      R[[i]] <- replace(R[[i]], P != (i + 1), 1)
    }
  }
  R
}

# This is a custom adaptation of function binsearch from package gtools
# (version 3.5.0) that returns the adjustment of the probability curves used
# in the function ampute.continuous in ampute.
bin.search <- function(fun, range = c(-8, 8), ..., target = 0,
                       lower = ceiling(min(range)),
                       upper = floor(max(range)),
                       maxiter = 100, showiter = FALSE) {
  lo <- lower
  hi <- upper
  counter <- 0
  val.lo <- round(fun(lo, ...), 3)
  val.hi <- round(fun(hi, ...), 3)
  sign <- if (val.lo > val.hi) -1 else 1
  if (target * sign < val.lo * sign) {
    outside.range <- TRUE
  } else if (target * sign > val.hi * sign) {
    outside.range <- TRUE
  } else {
    outside.range <- FALSE
  }
  while (counter < maxiter && !outside.range) {
    counter <- counter + 1
    if (hi - lo <= (1 / (10^3)) || lo < lower || hi > upper) {
      break
    }
    center <- round((hi - lo) / 2 + lo, 3)
    val <- round(fun(center, ...), 3)
    if (showiter) {
      cat("--------------\n")
      cat("Iteration #", counter, "\n")
      cat("lo=", lo, "\n")
      cat("hi=", hi, "\n")
      cat("center=", center, "\n")
      cat("fun(lo)=", val.lo, "\n")
      cat("fun(hi)=", val.hi, "\n")
      cat("fun(center)=", val, "\n")
    }
    if (val == target) {
      val.lo <- val.hi <- val
      lo <- hi <- center
      break
    }
    else if (sign * val < sign * target) {
      lo <- center
      val.lo <- val
    }
    else {
      hi <- center
      val.hi <- val
    }
    if (showiter) {
      cat("new lo=", lo, "\n")
      cat("new hi=", hi, "\n")
      cat("--------------\n")
    }
  }
  retval <- list(call = match.call(), numiter = counter)
  if (outside.range) {
    if (target * sign < val.lo * sign) {
      warning("The desired proportion of ", target, " is too small; ", val.lo, " is used instead.")
      retval$flag <- "Lower Boundary"
      retval$where <- lo
      retval$value <- val.lo
    }
    else {
      warning("The desired proportion of ", target, " is too large; ", val.hi, " is used instead.")
      retval$flag <- "Upper Boundary"
      retval$where <- hi
      retval$value <- val.hi
    }
  }
  else if (counter >= maxiter) {
    retval$flag <- "Maximum number of iterations reached"
    retval$where <- (lo + hi) / 2
    retval$value <- (val.lo + val.hi) / 2
  }
  else if (val.lo == target) {
    retval$flag <- "Found"
    retval$where <- lo
    retval$value <- val.lo
  }
  else if (val.hi == target) {
    retval$flag <- "Found"
    retval$where <- hi
    retval$value <- val.hi
  }
  else {
    retval$flag <- "Between Elements"
    retval$where <- (lo + hi) / 2
    retval$value <- (val.lo + val.hi) / 2
  }
  retval
}

''')

 patterns <- data.frame(X1 = c(0,1,1,0,1,1,0,1,1),
		X2 = c(1,1,1,1,1,1,1,1,1),
		X3 = c(1,1,1,1,1,1,1,1,1),
		X4 = c(1,1,1,1,1,1,1,1,1),
		X5 = c(1,1,1,1,1,1,1,1,1),
		X6 = c(1,1,1,1,1,1,1,1,1),
		X7 = c(1,1,1,1,1,1,1,1,1),
		X8 = c(1,1,1,1,1,1,1,1,1),
		X9 = c(1,1,1,1,1,1,1,1,1),
                stringsAsFactors = FALSE)